In [1]:
from nltk.tokenize import word_tokenize
import math

#Function to tokenise string/sentences.
def tokenize(line, tokenizer=word_tokenize):
    utf_line = line.lower()
    return [token for token in tokenizer(utf_line)]

In [2]:
tokenize("Seit damals ist er auf über 10.000 Punkte gestiegen.")

['seit',
 'damals',
 'ist',
 'er',
 'auf',
 'über',
 '10.000',
 'punkte',
 'gestiegen',
 '.']

In [3]:
DEVELOPMENT_DOCS = 'dataset/devel.docs' #Data file for IR engine development

DEVELOPMENT_QUERIES = 'dataset/devel.queries' #Data file containing queries in German

DEVELOPMENT_QREL = 'dataset/devel.qrel' #Data file containing a relevance score or query-doc pairs

BITEXT_ENG = 'dataset/bitext.en' #Bitext data file in English for translation engine and language model development

BITEXT_DE = 'dataset/bitext.de' #Bitext data file in German

NEWSTEST_ENG = 'dataset/newstest.en' #File for testing language model

In [4]:
import nltk
import re

stopwords = set(nltk.corpus.stopwords.words('english')) #converting stopwords to a set for faster processing in the future.
stemmer = nltk.stem.PorterStemmer() 

#Function to extract and tokenize terms from a document
def extract_and_tokenize_terms(doc):
    terms = []
    for token in tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are faster over sets than lists
            if not re.search(r'\d',token) and not re.search(r'[^A-Za-z-]',token): #Removing numbers and punctuations 
                #(excluding hyphenated words)
                terms.append(stemmer.stem(token.lower()))
    return terms

documents = {} #Dictionary to store documents with ids as keys.

In [5]:
#Reading each line in the file and storing it documents dictionary
f = open(DEVELOPMENT_DOCS)

for line in f:
    doc = line.split("\t")
    terms = extract_and_tokenize_terms(doc[1])
    documents[doc[0]] = terms
f.close()

In [6]:
documents['290'][:20] #To keep things short, we're only going to check out 20 tokens.

['name',
 'plural',
 'ae',
 'first',
 'letter',
 'vowel',
 'iso',
 'basic',
 'latin',
 'alphabet',
 'similar',
 'ancient',
 'greek',
 'letter',
 'alpha',
 'deriv',
 'upper',
 'case',
 'version',
 'consist']

In [7]:
#Building an inverted index for the documents

from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid) 

In [8]:
inverted_index['pizza']

{'121569',
 '16553',
 '212541',
 '228211',
 '261023',
 '265975',
 '276433',
 '64083',
 '69930',
 '72701',
 '73441',
 '74323'}

In [9]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Function to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [10]:
#Creating tf_idf matrix with said parameter values: k1 and b for all documents.
tf_idf = create_tf_idf(1.5,0.5)

In [11]:
#Function to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Function to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in query.split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count]   

In [12]:
retr_docs("Manchester United",5)

[('19961', 12.570768983660066),
 ('83266', 12.500411244644424),
 ('266959', 12.464229013042164),
 ('20206', 12.324367209127187),
 ('253314', 12.008594517514213)]

In [13]:
documents['19961'][:30]

['manchest',
 'unit',
 'manchest',
 'unit',
 'footbal',
 'club',
 'english',
 'profession',
 'footbal',
 'club',
 'base',
 'old',
 'trafford',
 'greater',
 'manchest',
 'play',
 'premier',
 'leagu',
 'found',
 'newton',
 'heath',
 'lyr',
 'footbal',
 'club',
 'club',
 'chang',
 'name',
 'manchest',
 'unit',
 'move']

In [14]:
#Calculating the unigram, bigram and trigram counts. 

f = open(BITEXT_ENG)

train_sentences = []

for line in f:
    train_sentences.append(tokenize(line))

f.close()    

#Function to mark the first occurence of words as unknown, for training.
def check_for_unk_train(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        unigram_counts[word] = 0
        return "UNK"

#Function to convert sentences for training the language model.    
def convert_sentence_train(sentence,unigram_counts):
    #<s1> and <s2> are sentinel tokens added to the start and end, for handling tri/bigrams at the start of a sentence.
    return ["<s1>"] + ["<s2>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s2>"]+ ["</s1>"]

#Function to obtain unigram, bigram and trigram counts.
def get_counts(sentences):
    trigram_counts = defaultdict(lambda: defaultdict(dict))
    bigram_counts = defaultdict(dict)
    unigram_counts = {}
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 2):
            trigram_counts[sentence[i]][sentence[i+1]][sentence[i+2]] = trigram_counts[sentence[i]][sentence[i+1]].get(sentence[i+2],0) + 1
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    unigram_counts["</s1>"] = unigram_counts["<s1>"]
    unigram_counts["</s2>"] = unigram_counts["<s2>"]
    bigram_counts["</s2>"]["</s1>"] = bigram_counts["<s1>"]["<s2>"]
    return unigram_counts, bigram_counts, trigram_counts

In [15]:
unigram_counts, bigram_counts,trigram_counts = get_counts(train_sentences)

In [16]:
#Constructing unigram model with 'add-k' smoothing
token_count = sum(unigram_counts.values())

#Function to convert unknown words for testing. 
#Words that don't appear in the training corpus (even if they are in the test corpus) are marked as UNK.
def check_for_unk_test(word,unigram_counts):
    if word in unigram_counts and unigram_counts[word] > 0:
        return word
    else:
        return "UNK"


def convert_sentence_test(sentence,unigram_counts):
    return ["<s1>"] + ["<s2>"] + [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s2>"]  + ["</s1>"]

#Returns the log probability of a unigram, with add-k smoothing. We're taking logs to avoid probability underflow.
def get_log_prob_addk(word,unigram_counts,k):
    return math.log((unigram_counts[word] + k)/ \
                    (token_count + k*len(unigram_counts)))

#Returns the log probability of a sentence.
def get_sent_log_prob_addk(sentence, unigram_counts,k):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_addk(word, unigram_counts,k) for word in sentence])


def calculate_perplexity_uni(sentences,unigram_counts, token_count, k):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += get_sent_log_prob_addk(sentence,unigram_counts,k)
    return math.exp(-total_log_prob/test_token_count)


f = open(NEWSTEST_ENG)

test_sents = []
for line in f:
    test_sents.append(tokenize(line))
f.close()

In [17]:
#Calculating the perplexity for different ks
ks = [0.0001,0.01,0.1,1,10]

for k in ks:
    print(str(k) +": " + str(calculate_perplexity_uni(test_sents,unigram_counts,token_count,k)))


0.0001: 631.5624344258267
0.01: 631.6372709016225
0.1: 632.3728621611093
1: 643.2578962510463
10: 814.7925245672897


In [18]:
#Calculating the N1/N paramaters for Trigrams/Bigrams/Unigrams in Katz-Backoff Smoothing

TRI_ONES = 0 #N1 for Trigrams
TRI_TOTAL = 0 #N for Trigrams

for twod in trigram_counts.values():
    for oned in twod.values():
        for val in oned.values():
            if val==1:
                TRI_ONES+=1 #Count of trigram seen once
            TRI_TOTAL += 1 #Count of all trigrams seen

BI_ONES = 0 #N1 for Bigrams
BI_TOTAL = 0 #N for Bigrams

for oned in bigram_counts.values():
    for val in oned.values():
        if val==1:
            BI_ONES += 1 #Count of bigram seen once
        BI_TOTAL += 1 #Count of all bigrams seen
        
UNI_ONES = list(unigram_counts.values()).count(1)
UNI_TOTAL = len(unigram_counts)

In [19]:
#Constructing trigram model with backoff smoothing

TRI_ALPHA = TRI_ONES/TRI_TOTAL #Alpha parameter for trigram counts
    
BI_ALPHA = BI_ONES/BI_TOTAL #Alpha parameter for bigram counts

UNI_ALPHA = UNI_ONES/UNI_TOTAL
    
def get_log_prob_back(sentence,i,unigram_counts,bigram_counts,trigram_counts,token_count):
    if trigram_counts[sentence[i-2]][sentence[i-1]].get(sentence[i],0) > 0:
        return math.log((1-TRI_ALPHA)*trigram_counts[sentence[i-2]][sentence[i-1]].get(sentence[i])/bigram_counts[sentence[i-2]][sentence[i-1]])
    else:
        if bigram_counts[sentence[i-1]].get(sentence[i],0)>0:
            return math.log(TRI_ALPHA*((1-BI_ALPHA)*bigram_counts[sentence[i-1]][sentence[i]]/unigram_counts[sentence[i-1]]))
        else:
            return math.log(TRI_ALPHA*BI_ALPHA*(1-UNI_ALPHA)*((unigram_counts[sentence[i]]+0.0001)/(token_count+(0.0001)*len(unigram_counts)))) 
        
        
def get_sent_log_prob_back(sentence, unigram_counts, bigram_counts,trigram_counts, token_count):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_prob_back(sentence,i, unigram_counts,bigram_counts,trigram_counts,token_count) for i in range(2,len(sentence))])


def calculate_perplexity_tri(sentences,unigram_counts,bigram_counts,trigram_counts, token_count):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += get_sent_log_prob_back(sentence,unigram_counts,bigram_counts,trigram_counts,token_count)
    return math.exp(-total_log_prob/test_token_count)

In [20]:
#Calculating the perplexity 
calculate_perplexity_tri(test_sents,unigram_counts,bigram_counts,trigram_counts,token_count)

463.6271760983476

In [21]:
#Creating lists of English and German sentences from bitext.

from nltk.translate import IBMModel1
from nltk.translate import AlignedSent, Alignment

eng_sents = []
de_sents = []

f = open(BITEXT_ENG)
for line in f:
    terms = tokenize(line)
    eng_sents.append(terms)
f.close()

f = open(BITEXT_DE)
for line in f:
    terms = tokenize(line)
    de_sents.append(terms)
f.close()

In [22]:
#Zipping together the bitexts for easier access
paral_sents = list(zip(eng_sents,de_sents))

In [23]:
#Building English to German translation table for words (Backward alignment)
eng_de_bt = [AlignedSent(E,G) for E,G in paral_sents]
eng_de_m = IBMModel1(eng_de_bt, 5)

In [24]:
#Building German to English translation table for words (Backward alignment)
de_eng_bt = [AlignedSent(G,E) for E,G in paral_sents]
de_eng_m = IBMModel1(de_eng_bt, 5)

In [25]:
#Script below to combine alignments using set intersections
combined_align = []

for i in range(len(eng_de_bt)):

    forward = {x for x in eng_de_bt[i].alignment}
    back_reversed = {x[::-1] for x in de_eng_bt[i].alignment}
    
    combined_align.append(forward.intersection(back_reversed))

In [26]:
#Creating German to English dictionary with occurence count of word pairs
de_eng_count = defaultdict(dict)

for i in range(len(de_eng_bt)):
    for item in combined_align[i]:
        de_eng_count[de_eng_bt[i].words[item[1]]][de_eng_bt[i].mots[item[0]]] =  de_eng_count[de_eng_bt[i].words[item[1]]].get(de_eng_bt[i].mots[item[0]],0) + 1

In [27]:
#Creating a English to German dict with occ count of word pais
eng_de_count = defaultdict(dict)

for i in range(len(eng_de_bt)):
    for item in combined_align[i]:
        eng_de_count[eng_de_bt[i].words[item[0]]][eng_de_bt[i].mots[item[1]]] =  eng_de_count[eng_de_bt[i].words[item[0]]].get(eng_de_bt[i].mots[item[1]],0) + 1

In [28]:
#Creating German to English table with word translation probabilities
de_eng_prob = defaultdict(dict)

for de in de_eng_count.keys():
    for eng in de_eng_count[de].keys():
        de_eng_prob[de][eng] = de_eng_count[de][eng]/sum(de_eng_count[de].values())

In [29]:
#Creating English to German dict with word translation probabilities 
eng_de_prob = defaultdict(dict)

for eng in eng_de_count.keys():
    for de in eng_de_count[eng].keys():
        eng_de_prob[eng][de] = eng_de_count[eng][de]/sum(eng_de_count[eng].values())

In [46]:
#Examples of translating individual words from German to English
print(de_eng_prob['frage'])

print(de_eng_prob['handlung'])

print(de_eng_prob['haus'])

print(de_eng_prob['die'])

{'question': 1.0}
{'spans': 0.5, 'side': 0.5}
{'house': 0.625, 'charity': 0.125, 'hospitalized': 0.125, 'offset': 0.125}
{'the': 1.0}


In [31]:
#Building noisy channel translation model
def de_eng_noisy(german):
    noisy={}
    for eng in de_eng_prob[german].keys():
        noisy[eng] = eng_de_prob[eng][german]+ get_log_prob_addk(eng,unigram_counts,0.0001)
    return noisy

In [32]:
#Test block to check alignments
print(de_eng_noisy('vater'))
print(de_eng_noisy('haus'))
print(de_eng_noisy('das'))
print(de_eng_noisy('entschuldigung'))

{}
{'house': -8.153224281882322, 'charity': -11.032065847914977, 'hospitalized': -11.319739587276473, 'offset': -11.225188029412386}
{'this': -4.991400947456017, 'that': -4.857303935975769, 'is': -4.278625574868814, 'the': -3.0973206970208214, 'for': -5.074260930518287, 'of': -3.88129895540149}
{'pledging': -21.128725580698557}


In [45]:
print(eng_de_prob['sorry'])
print(eng_de_prob['the'])

{'bereue': 1.0}
{'den': 0.030258662762323085, 'die': 0.7484952009110135, 'der': 0.2010736944851147, 'das': 0.014803969415975272, 'des': 0.002277533756303888, 'dem': 0.0016268098259313486, 'im': 0.0014641288433382138}


In [34]:
#Translating first 5 queries into English

#Function for direct translation
def de_eng_direct(query):
    query_english = [] 
    query_tokens = tokenize(query)
    
    for token in query_tokens:
        try:
            query_english.append(max(de_eng_prob[token], key=de_eng_prob[token].get))
        except:
            query_english.append(token) #Returning the token itself when it cannot be found in the translation table.
            #query_english.append("NA") 
    
    return " ".join(query_english)

#Function for noisy channel translation
def de_eng_noisy_translate(query):  
    query_english = [] 
    query_tokens = tokenize(query)
    
    for token in query_tokens:
        try:
            query_english.append(max(de_eng_noisy(token), key=de_eng_noisy(token).get))
        except:
            query_english.append(token) #Returning the token itself when it cannot be found in the translation table.
            #query_english.append("NA") 
    
    return " ".join(query_english)
            
f = open(DEVELOPMENT_QUERIES)

lno = 0
plno = 0

#Also building a dictionary of query ids and query content (only for the first 100s)
german_qs = {}

test_query_trans_sents = [] #Building a list for perplexity checks.

for line in f:
    lno+=1
    query_id = line.split('\t')[0]
    query_german = line.split('\t')[1]  
    
    german_qs[query_id] = query_german.strip()
    
    translation = str(de_eng_noisy_translate(query_german))
    if plno<5:
        print(query_id + "\n" + "German: " + str(query_german) + "\n" + "English: " + translation +"\n\n")
        plno+=1
    test_query_trans_sents.append(translation)
    if lno==100:
        break

f.close()

82
German: der ( von engl . action : tat , handlung , bewegung ) ist ein filmgenre des unterhaltungskinos , in welchem der fortgang der äußeren handlung von zumeist spektakulär inszenierten kampf - und gewaltszenen vorangetrieben und illustriert wird .

English: the ( , leninism . action : rattling , side , movement ) is a filmgenre the unterhaltungskinos , in paulson the fortgang the trumpet side , zumeist spektakulär inszenierten fight - and gewaltszenen annan and illustriert is .


116
German: die ( einheitenzeichen : u für unified atomic mass unit , veraltet amu für atomic mass unit ) ist eine maßeinheit der masse .

English: the ( einheitenzeichen : u for unified atomic manipulation unit , regime amu for atomic manipulation unit ) is a befuddled the masse .


240
German: der von lateinisch actualis , " wirklich " , auch aktualitätsprinzip , uniformitäts - oder gleichförmigkeitsprinzip , englisch uniformitarianism , ist die grundlegende wissenschaftliche methode in der .

English: 

In [35]:
#Building a dictionary for queryids and relevant document ids
qrel = defaultdict(list)

f = open(DEVELOPMENT_QREL)

for line in f:
    item = line.split('\t')
    qrel[item[0]].append(item[2])
    
f.close()

In [36]:
#Single function to retreive documents for a German query
def trans_retr_docs(german_query,no_of_results,translation_function):
    
    trans_query = " ".join(extract_and_tokenize_terms(translation_function(german_query)))
    return [item[0] for item in retr_docs(trans_query,no_of_results)] #Retriving 100 documents

#Calculating the map score
def calc_map(no_of_results,translation_function):
    
    average_precision = []
    
    for gq in german_qs.keys():
        
        relevant_docs = qrel[gq]
        incremental_precision = []
        resulting_docs = trans_retr_docs(german_qs[gq],no_of_results,translation_function)
        
        total_counter = 0
        true_positive_counter = 0
        
        for doc in resulting_docs:
            total_counter+=1
            if doc in relevant_docs:
                true_positive_counter += 1
                incremental_precision.append(true_positive_counter/total_counter)
        
        #For no relevant retreivals, the average precision will be considered 0.
        try:
            average_precision.append(sum(incremental_precision)/len(incremental_precision))
        except:
            average_precision.append(0)
        
    return (sum(average_precision)/len(average_precision))

In [37]:
#Printing the map score for direct translations
print(calc_map(100,de_eng_direct))

0.23816698096268418


In [38]:
#Printing the map score for noisy channel translations
print(calc_map(100,de_eng_noisy_translate))

0.25474638545708655
